In [3]:
import numbers
import unittest

In [10]:
class IntegerField:
    def __init__(self, min_, max_):
        self._min = min_ 
        self._max = max_
    
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f"{self.prop_name} must be an integer value.")
        if value < self._min:
             raise ValueError(f"{self.prop_name} must be >= {self._min}.")
        if value > self._max:
             raise ValueError(f"{self.prop_name} must be <= {self._max}.")
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

tests

In [11]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    

In [12]:
class TestIntegerField(unittest.TestCase):
    
    class Person:
        age = IntegerField(0,10)
        
    def test_set_age_ok(self):
        p = self.Person()
        p.age = 0
        self.assertEqual(0, p.age)

In [13]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


let's try to override the discriptor attribute

In [16]:
class TestIntegerField(unittest.TestCase):
    
    class Person:
        age = IntegerField(0,10)
        
    def test_set_age_ok(self):
        
        self.Person.age = IntegerField(5, 10)
        p = self.Person()
        p.age = 5
        self.assertEqual(5, p.age)
# will get an error - __set_name__ will not get called, 
#because we have created instance of the descriptor outside Person class 

In [17]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ERROR

ERROR: test_set_age_ok (__main__.TestIntegerField)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-16-533f86fad74b>", line 10, in test_set_age_ok
    p.age = 5
  File "<ipython-input-10-bf622fced744>", line 16, in __set__
    instance.__dict__[self.prop_name] = value
AttributeError: 'IntegerField' object has no attribute 'prop_name'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


we can patch to get rid of the error:

In [20]:
class TestIntegerField(unittest.TestCase):
    
    class Person:
        age = IntegerField(0,10)
        
    def test_set_age_ok(self):
        
        self.Person.age = IntegerField(5, 10)
        self.Person.age.__set_name__(self.Person, "age")
        p = self.Person()
        p.age = 5
        self.assertEqual(5, p.age)

In [21]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


let's break it out

In [29]:
class TestIntegerField(unittest.TestCase):
    
    class Person:
        age = IntegerField(0,10)
        
    def create_person(self, min_, max_):
        self.Person.age = IntegerField(min_, max_)
        self.Person.age.__set_name__(self.Person, "age")
        return self.Person()
        
    def test_set_age_ok(self):
              
        p = self.create_person(5,10)
        p.age = 5
        self.assertEqual(5, p.age)

In [30]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


#### without patching

___________________________

we can create classes using functional approach

In [32]:
class Person:
    a = 10
type(Person)

type

type is a class in python, it means that it is callable

In [33]:
type()

TypeError: type() takes 1 or 3 arguments

using type and its constructor we can create classes

In [34]:
Person = type("Person",(), {"a":11})
# name of the class,inheritance, dict with class attributes

In [35]:
type(Person)

type

In [36]:
Person.__dict__

mappingproxy({'a': 11,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [37]:
Person.a

11

In [38]:
p = Person()
p

_____________________________

In [41]:
class TestIntegerField(unittest.TestCase):
    
   
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"age":IntegerField(min_,max_)})
        return obj()
        
        
    def test_set_age_ok(self):
              
        p = self.create_person_instance(5,10)
        p.age = 5
        self.assertEqual(5, p.age)

In [42]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


let's expand our tests

In [45]:
class TestIntegerField(unittest.TestCase):
    
   
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"age":IntegerField(min_,max_)})
        return obj()
        
        
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, max_+1)
        
        for i, value in enumerate(valid_values):
            p.age = value
            self.assertEqual(value, p.age)

In [46]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [51]:
class TestIntegerField(unittest.TestCase):
    
   
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"age":IntegerField(min_,max_)})
        return obj()
       
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, max_+1)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
            
    def test_set_age_invalid(self):
        min_ = -10
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        
        bad_values = list(range(min_ -5, min_))
        bad_values += list(range(max_ +1, max_+5))
        bad_values += [10.5, 1+0j, "abc", (1,2)]
        for i, value in enumerate(bad_values):
             with self.subTest(test_number=i):
                    with self.assertRaises(ValueError):
                        p.age = value
                        
    def test_class_get(self):
        p = self.create_person_instance(0, 0)
        p_class = type(p)
        self.assertIsInstance(p_class.age, IntegerField)


In [52]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


let's make min and max optional

first write tests

In [53]:
class TestIntegerField(unittest.TestCase):
    
   
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"age":IntegerField(min_,max_)})
        return obj()
       
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, max_+1)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
            
    def test_set_age_invalid(self):
        min_ = -10
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        
        bad_values = list(range(min_ -5, min_))
        bad_values += list(range(max_ +1, max_+5))
        bad_values += [10.5, 1+0j, "abc", (1,2)]
        for i, value in enumerate(bad_values):
             with self.subTest(test_number=i):
                    with self.assertRaises(ValueError):
                        p.age = value
                        
    def test_class_get(self):
        p = self.create_person_instance(0, 0)
        p_class = type(p)
        self.assertIsInstance(p_class.age, IntegerField)
        
        
    def test_set_age_min_only(self):
        min_ = 5
        max_ = None      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, min_+100, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
                
    def test_set_age_max_only(self):
        min_ = None
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(max_-100, max_, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
                
    def test_set_age_no_limits(self):
        min_ = None
        max_ = None      
        p = self.create_person_instance(min_, max_)
        valid_values = range(-100, 100, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
            


In [54]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_max_only (__main__.TestIntegerField) ... test_set_age_min_only (__main__.TestIntegerField) ... test_set_age_no_limits (__main__.TestIntegerField) ... test_set_age_ok (__main__.TestIntegerField) ... ok

ERROR: test_set_age_max_only (__main__.TestIntegerField) (test_number=0)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-53-c0b4dd6643b6>", line 59, in test_set_age_max_only
    p.age = value
  File "<ipython-input-10-bf622fced744>", line 12, in __set__
    if value < self._min:
TypeError: '<' not supported between instances of 'int' and 'NoneType'

ERROR: test_set_age_max_only (__main__.TestIntegerField) (test_number=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-53-c0b4dd6643b6>", line 59, in test_set_age_

In [55]:
class IntegerField:
    def __init__(self, min_=None, max_=None):
        self._min = min_ 
        self._max = max_
    
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f"{self.prop_name} must be an integer value.")
        if self._min is not None and value < self._min:
             raise ValueError(f"{self.prop_name} must be >= {self._min}.")
        if self._max is not None and value > self._max:
             raise ValueError(f"{self.prop_name} must be <= {self._max}.")
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [56]:
class TestIntegerField(unittest.TestCase):
    
   
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"age":IntegerField(min_,max_)})
        return obj()
       
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, max_+1)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
            
    def test_set_age_invalid(self):
        min_ = -10
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        
        bad_values = list(range(min_ -5, min_))
        bad_values += list(range(max_ +1, max_+5))
        bad_values += [10.5, 1+0j, "abc", (1,2)]
        for i, value in enumerate(bad_values):
             with self.subTest(test_number=i):
                    with self.assertRaises(ValueError):
                        p.age = value
                        
    def test_class_get(self):
        p = self.create_person_instance(0, 0)
        p_class = type(p)
        self.assertIsInstance(p_class.age, IntegerField)
        
        
    def test_set_age_min_only(self):
        min_ = 5
        max_ = None      
        p = self.create_person_instance(min_, max_)
        valid_values = range(min_, min_+100, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
                
    def test_set_age_max_only(self):
        min_ = None
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_values = range(max_-100, max_, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
                
    def test_set_age_no_limits(self):
        min_ = None
        max_ = None      
        p = self.create_person_instance(min_, max_)
        valid_values = range(-100, 100, 10)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                p.age = value
                self.assertEqual(value, p.age)
            


In [57]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_max_only (__main__.TestIntegerField) ... ok
test_set_age_min_only (__main__.TestIntegerField) ... ok
test_set_age_no_limits (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


In [58]:
class CharField:
    def __init__(self, min_=None, max_=None):
        min_ = min_ or 0
        min_ = max(0, min_)
        self._min = min_ 
        self._max = max_
    
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f"{self.prop_name} must be a string.")
        if self._min is not None and len(value) < self._min:
             raise ValueError(f"{self.prop_name} must be >= {self._min} chars.")
        if self._max is not None and len(value) > self._max:
             raise ValueError(f"{self.prop_name} must be <= {self._max} chars.")
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [59]:
class Person:
    name = CharField(1, 10)

In [60]:
p = Person()
p.name = ""

ValueError: name must be >= 1 chars.

In [61]:
p.name = "alex"

In [62]:
class Person:
    name = CharField(1)

In [63]:
p = Person()
p.name = "alex"*10

In [64]:
class Person:
    name = CharField(max_ = 1)

In [65]:
p = Person()
p.name = "alex"*10

ValueError: name must be <= 1 chars.

In [66]:
p.name = "j"

tests

In [73]:
class TestCharField(unittest.TestCase):
    
    @staticmethod    
    def create_person_instance( min_, max_):
        obj = type("Person",(), {"name":CharField(min_,max_)})
        return obj()
       
        
    def test_set_name_ok(self):
        min_ = 1
        max_ = 10      
        p = self.create_person_instance(min_, max_)
        valid_length = range(min_, max_+1)
        
        for i, length in enumerate(valid_length):
            value = "a" * length
            with self.subTest(test_number=i):
                p.name = value
                self.assertEqual(value, p.name)
            

                        
    def test_class_get(self):
        p = self.create_person_instance(0, 0)
        p_class = type(p)
        self.assertIsInstance(p_class.name, CharField)
        

In [74]:
run_tests(TestCharField)

test_class_get (__main__.TestCharField) ... ok
test_set_name_ok (__main__.TestCharField) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


#### let's use inheritance

In [75]:
class BaseValidator:
    def __init__(self, min_=None, max_=None):
        self._min = min_ 
        self._max = max_
    
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def validate(self, value):
        pass
        
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
        
class CharField(BaseValidator):
    def __init__(self, min_=None, max_=None):
        min_ = max(min_ or 0, 0)
        super().__init__(min_, max_)
        
    def validate(self, value):
        if not isinstance(value, str):
            raise ValueError(f"{self.prop_name} must be a string.")
        if self._min is not None and len(value) < self._min:
             raise ValueError(f"{self.prop_name} must be >= {self._min} chars.")
        if self._max is not None and len(value) > self._max:
             raise ValueError(f"{self.prop_name} must be <= {self._max} chars.")
       
       
class IntegerField(BaseValidator):
 
    def validate(self, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f"{self.prop_name} must be an integer value.")
        if self._min is not None and value < self._min:
             raise ValueError(f"{self.prop_name} must be >= {self._min}.")
        if self._max is not None and value > self._max:
             raise ValueError(f"{self.prop_name} must be <= {self._max}.")
  

In [76]:
run_tests(TestCharField)

test_class_get (__main__.TestCharField) ... ok
test_set_name_ok (__main__.TestCharField) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [77]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_max_only (__main__.TestIntegerField) ... ok
test_set_age_min_only (__main__.TestIntegerField) ... ok
test_set_age_no_limits (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.012s

OK
